# Zero-Waste Market Data Exploration

## Sales Summary

### Annual Sales Volume

#### Sales, Pounds Sold, Transaction, Customer and Product Counts, and AOV by Year

In [1]:
SELECT 
    YEAR(o.OrderDate) AS Year, 
    SUM(l.SaleAmount) AS Sales,
    SUM(l.PoundsSold) AS PoundsSold, 
    COUNT(DISTINCT o.OrderID) As TransactionCount,
    COUNT(DISTINCT o.CustomerID) AS KnownCustomerCount,
    COUNT(DISTINCT l.ProductID) AS UniqueProductCount,
    COUNT(DISTINCT l.SalesOrderLineID) AS TransactionLineCount,
    SUM(l.SaleAmount)/ COUNT(DISTINCT o.OrderID) AS AvgOrderValue

FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 

GROUP BY ROLLUP (YEAR(o.OrderDate))
ORDER BY YEAR(o.OrderDate) 

(4 rows affected)

Total execution time: 00:00:01.935

Year,Sales,PoundsSold,TransactionCount,KnownCustomerCount,UniqueProductCount,TransactionLineCount,AvgOrderValue
NULL,1308799.38,200821.7646,41450,3932,1006,223151,31.575377
2019,277348.76,37539.4033,11776,1704,673,52856,23.552034
2020,494122.18,76139.6552,14648,1769,635,83723,33.733081
2021,537328.44,87142.7061,15026,2137,582,86572,35.759912


#### Sales Per Square Foot, Annualized by Location

The retail store opened in 2019 and relocated mid-2021.  This query calculates annualized sales per square foot by location, adjusting for partial years

In [1]:
-- AnnualizedSales per square foot

SELECT 
    o.LocationID,
    YEAR(o.OrderDate) AS Year,
    SUM(l.SaleAmount) AS Sales,
    DATEDIFF(Day,MIN(o.OrderDate), MAX(o.OrderDate))+1 AS DaysInPeriod,

    -- calculate annualized sales by dividing sales by number of days in period and multiplying by 365
    SUM(l.SaleAmount)/(DATEDIFF(Day,MIN(o.OrderDate), MAX(o.OrderDate))+1 )*365 AS AnnualizedSales,

    MAX(dl.Sqft) AS Sqft,

    -- Divide Annualized Sales by Location SqFt
    (SUM(l.SaleAmount)/(DATEDIFF(Day,MIN(o.OrderDate), MAX(o.OrderDate))+1 )*365) / MAX(dl.Sqft) AS SalesPerSqft

FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID
    LEFT JOIN DimLocation as dl
    ON o.LocationID = dl.LocationID

GROUP BY o.LocationID, YEAR(o.OrderDate)
ORDER BY Year, LocationID

(4 rows affected)

Total execution time: 00:00:00.456

LocationID,Year,Sales,DaysInPeriod,AnnualizedSales,Sqft,SalesPerSqft
1,2019,277348.76,331,305837.756265,800,382.297195
1,2020,494122.18,366,492772.119365,800,615.965149
1,2021,411605.55,261,575616.956670,800,719.521195
2,2021,125722.89,100,458888.548500,1200,382.407123


### Monthly Sales Trend
#### Total Monthly Sales and Average Daily Sales by Month

In [3]:
SELECT 
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales, --Total Montly Sales / Number of Days in Month
    COUNT(DISTINCT o.OrderID) AS TransactionCount,
    SUM(l.PoundsSold) AS Pounds

FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 

GROUP BY EOMONTH(o.OrderDate)
ORDER BY EOMONTH(o.OrderDate)   

(35 rows affected)

Total execution time: 00:00:00.604

TransactionMonth,Sales,AvgDailySales,TransactionCount,Pounds
2019-02-28,14964.79,534.460000,749,2085.1035
2019-03-31,20949.45,675.790000,1003,2907.3225
2019-04-30,17287.15,576.240000,899,2314.4868
2019-05-31,24424.98,787.900000,1184,3272.4228
2019-06-30,27454.37,915.150000,1137,3571.0753
2019-07-31,27423.94,884.640000,1120,3702.0144
2019-08-31,26306.96,848.610000,1095,3793.7712
2019-09-30,28842.81,961.430000,1109,3928.8019
2019-10-31,29368.16,947.360000,1135,4065.9887
2019-11-30,29010.85,967.030000,1153,3831.1585


#### Top 5 Sales Months by Revenue

In [2]:
SELECT TOP(5)
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS MonthlySales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales -- Total Monthly Sales / Number of Days in month

FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 

GROUP BY EOMONTH(o.OrderDate) --Group by Month
ORDER BY MonthlySales DESC     

(5 rows affected)

Total execution time: 00:00:00.190

TransactionMonth,MonthlySales,AvgDailySales
2021-01-31,55312.48,1784.270000
2021-02-28,54057.65,1930.630000
2020-12-31,53039.45,1710.950000
2021-04-30,52109.81,1736.990000
2021-03-31,51043.98,1646.580000


#### Monthly Sales - MoM, YoY % Change - Using Window LAG function and CTEs


In [5]:
--CTE to create a table of Total Sales by Month 
WITH MonthlySales AS(
SELECT 
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales  --Monthly Sales / number of days in the month
FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 
GROUP BY EOMONTH(o.OrderDate)
)

-- Compare Each Month to the Prior Month, and the Prior Year
SELECT  
    TransactionMonth,
    Sales,
    
    -- Monthly Sales Percent Change from Prior Month = Current month sales / Prior month sales - 1 
    ROUND(Sales/ 
        (LAG(Sales) -- use the LAG Window function with no offset, which defaults to the prior record (offset of 1)
            OVER ( ORDER BY TransactionMonth))-1,4) -- subtract 1 from the ratio of current/prior month sales to calculate the percentage change
            AS SalesPctChgMoM,  
    
    -- Monthly Sales % Change from Same Month Prior Year = Current Month/ Lag 12 month Sales -1
    ROUND(Sales/ 
    (LAG(Sales,12) -- Use the Lag Function with offset of 12 to retrive the prior year sales (assuming continuous sales reported in all months)
        OVER (ORDER BY TransactionMonth))-1,4) -- subtract 1 from the ratio and round to 4 decimals
        AS SalesPctChgYoY, 
    
    AvgDailySales,
    
    -- Avg Daily Sales % change from Prior Month = Avg Daily Sales Current Month / ADS Prior Month-1 
    ROUND((AvgDailySales/
    LAG(AvgDailySales) OVER(ORDER BY TransactionMonth))-1,4) AS AvgDailyPctChgMoM,

    -- Avg Daily Sales % change from Prior Year = Avg Daily Sales Current Month / ADS Prior Year-1 
    -- Assumes sales in all consecutive months
    ROUND((AvgDailySales/
    LAG(AvgDailySales,12) OVER(ORDER BY TransactionMonth))-1,4) AS AvgDailyPctChgYoY

FROM MonthlySales

(35 rows affected)

Total execution time: 00:00:00.352

TransactionMonth,Sales,SalesPctChgMoM,SalesPctChgYoY,AvgDailySales,AvgDailyPctChgMoM,AvgDailyPctChgYoY
2019-02-28,14964.79,NULL,NULL,534.460000,NULL,NULL
2019-03-31,20949.45,0.399900,NULL,675.790000,0.264400,NULL
2019-04-30,17287.15,-0.174800,NULL,576.240000,-0.147300,NULL
2019-05-31,24424.98,0.412900,NULL,787.900000,0.367300,NULL
2019-06-30,27454.37,0.124000,NULL,915.150000,0.161500,NULL
2019-07-31,27423.94,-0.001100,NULL,884.640000,-0.033300,NULL
2019-08-31,26306.96,-0.040700,NULL,848.610000,-0.040700,NULL
2019-09-30,28842.81,0.096400,NULL,961.430000,0.132900,NULL
2019-10-31,29368.16,0.018200,NULL,947.360000,-0.014600,NULL
2019-11-30,29010.85,-0.012200,NULL,967.030000,0.020800,NULL


#### Three Month Moving Average Monthly Sales - Using Window Function

In [6]:
--CTE to create a table of Total Sales by Month and by Product Category
WITH MonthlySales AS(
SELECT 
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales
FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 
GROUP BY EOMONTH(o.OrderDate)
)

-- Caculate a rolling three month average of sales by category
SELECT  
    TransactionMonth,
    SUM(Sales) TotalSales,    
    AVG(SUM(Sales)) OVER(
        ORDER BY TransactionMonth -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMonthAvgSales

FROM MonthlySales
GROUP BY TransactionMonth
ORDER BY TransactionMonth

(35 rows affected)

Total execution time: 00:00:00.333

TransactionMonth,TotalSales,MovingThreeMonthAvgSales
2019-02-28,14964.79,14964.790000
2019-03-31,20949.45,17957.120000
2019-04-30,17287.15,17733.796666
2019-05-31,24424.98,20887.193333
2019-06-30,27454.37,23055.500000
2019-07-31,27423.94,26434.430000
2019-08-31,26306.96,27061.756666
2019-09-30,28842.81,27524.570000
2019-10-31,29368.16,28172.643333
2019-11-30,29010.85,29073.940000


#### Three Month Moving Average Monthly Sales by Product Category - Using Window Function with Partition

Trailing Three Month Average Sales by Product Category

In [7]:
--CTE to create a table of Total Sales by Month and by Product Category
WITH MonthlySales AS(
SELECT 
    c.Category AS Category,
    EOMONTH(o.OrderDate) AS TransactionMonth, --Use the last day of the month to represent the Month. Since all data is historical do not need to account for midmonth data
    ROUND(SUM(l.SaleAmount),2) AS Sales, --Sum the Monthly Sales
    ROUND(SUM(l.SaleAmount)/DAY(EOMONTH(o.OrderDate)),2) AS AvgDailySales
FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID 
    INNER JOIN DimProducts AS p
    ON l.ProductID = p.ProductID
    INNER JOIN DimSubcategories AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    INNER JOIN DimCategories AS c
    ON sc.CategoryID = c.CategoryID
GROUP BY Category, EOMONTH(o.OrderDate)
)

-- Caculate a rolling three month average of sales by category
SELECT  
    Category,
    TransactionMonth,
    SUM(Sales) TotalSales,    
    AVG(SUM(Sales)) OVER(
        PARTITION BY Category  -- Divide the data by Category - the avg will be calcuated separately for each category
        ORDER BY TransactionMonth -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMonthAvgSales

FROM MonthlySales
WHERE Category IN ('Grocery','Body Care','Local Love')
GROUP BY
    Category,
    TransactionMonth
ORDER BY Category, TransactionMonth


(105 rows affected)

Total execution time: 00:00:00.570

Category,TransactionMonth,TotalSales,MovingThreeMonthAvgSales
Body Care,2019-02-28,2624.92,2624.920000
Body Care,2019-03-31,2544.82,2584.870000
Body Care,2019-04-30,2429.24,2532.993333
Body Care,2019-05-31,3537.36,2837.140000
Body Care,2019-06-30,3367.33,3111.310000
Body Care,2019-07-31,3557.08,3487.256666
Body Care,2019-08-31,3914.50,3612.970000
Body Care,2019-09-30,3686.71,3719.430000
Body Care,2019-10-31,3707.63,3769.613333
Body Care,2019-11-30,4093.80,3829.380000


## Product Analysis

#### Top Selling Product Categories and % of Total Revenue

In [13]:
SELECT 
    c.Category,
    SUM(l.SaleAmount) AS TotalSales,
    SUM(l.SaleAmount)/
        (SELECT SUM(l.SaleAmount)
            FROM SalesOrderLines AS l) AS PctOfTotalSales,
    SUM(l.PoundsSold) AS Pounds,
    SUM(l.PoundsSold)/ (SELECT SUM(PoundsSold) FROM SalesOrderLines) As PctofTotalPounds
        
FROM SalesOrderLines AS l
    INNER JOIN DimProducts AS p
    ON l.ProductID = p.ProductID
    INNER JOIN DimSubcategories AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    INNER JOIN DimCategories AS c
    ON sc.CategoryID = c.CategoryID

GROUP BY ROLLUP(c.Category)
ORDER BY SUM(l.SaleAmount) DESC

(14 rows affected)

Total execution time: 00:00:00.280

Category,TotalSales,PctOfTotalSales,Pounds,PctofTotalPounds
NULL,1308799.38,1.000000,200821.7646,1.000000
Grocery,601408.50,0.459511,113692.8352,0.566138
Local Love,247621.71,0.189197,22297.8726,0.111033
Body Care,175967.19,0.134449,18400.5515,0.091626
Cleaning Supply,102554.17,0.078357,34619.2360,0.172387
Whls_Grocery,54745.46,0.041828,158.9267,0.000791
Accessories,48086.01,0.036740,0.0000,0.000000
Containers,32241.63,0.024634,0.0000,0.000000
Fresh Produce,31740.42,0.024251,11591.3426,0.057719
Grab & Go,8398.09,0.006416,0.0000,0.000000


#### Top Selling Product Subcategories and % of Total Revenue

In [14]:
SELECT TOP (10)
    sc.Subcategory,
    c.Category,
    SUM(l.SaleAmount) AS TotalSales,
    SUM(l.SaleAmount)/
        (SELECT SUM(l.SaleAmount)
            FROM SalesOrderLines AS l) AS PctTotalSales,
    SUM(l.PoundsSold) AS PoundsSold,
    SUM(l.PoundsSold) / (SELECT SUM(PoundsSold) FROM SalesOrderLines) AS PctTotalPounds

FROM SalesOrderLines AS l
    INNER JOIN DimProducts AS p
    ON l.ProductID = p.ProductID
    INNER JOIN DimSubcategories AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    INNER JOIN DimCategories AS c
    ON sc.CategoryID = c.CategoryID

GROUP BY sc.Subcategory, c.Category
ORDER BY SUM(l.SaleAmount) DESC

(10 rows affected)

Total execution time: 00:00:00.302

Subcategory,Category,TotalSales,PctTotalSales,PoundsSold,PctTotalPounds
Nuts,Grocery,98596.61,0.075333,10184.2660,0.050712
Body Cleansing,Body Care,70617.06,0.053955,9056.6701,0.045098
Seeds And Grains,Grocery,67660.29,0.051696,22878.4230,0.113924
Oils,Grocery,66665.40,0.050936,9088.6121,0.045257
Laundry Cleaners,Cleaning Supply,54217.30,0.041425,15378.9517,0.076580
Dried Fruits,Grocery,42389.19,0.032387,4954.7091,0.024672
Sweeteners,Grocery,42159.78,0.032212,10885.4690,0.054204
Kombucha,Local Love,41918.87,0.032028,9299.1563,0.046305
Hair Care,Body Care,40704.96,0.031100,4818.4105,0.023993
Dairy,Grocery,39807.90,0.030415,290.0458,0.001444


#### Top Selling Products and % of Total Revenue

In [16]:
SELECT TOP(20)
    p.ProductName,
    sc.Subcategory,
    c.Category,
    SUM(l.SaleAmount) AS TotalSales,
    
    SUM(l.SaleAmount)/
        (SELECT SUM(l.SaleAmount)
            FROM SalesOrderLines AS l) AS PctOfTotal,

    SUM(l.PoundsSold) AS PoundsSold,

    SUM(l.PoundsSold)/
        (SELECT SUM(l.PoundsSold)
            FROM SalesOrderLines AS l) AS PctOfTotalPounds
        

FROM SalesOrderLines AS l
    INNER JOIN DimProducts AS p
    ON l.ProductID = p.ProductID
    LEFT JOIN DimSubcategories AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    LEFT JOIN DimCategories AS c
    ON sc.CategoryID = c.CategoryID

GROUP BY p.ProductName, sc.Subcategory, c.Category
ORDER BY SUM(l.SaleAmount) DESC    


(20 rows affected)

Total execution time: 00:00:00.336

ProductName,Subcategory,Category,TotalSales,PctOfTotal,PoundsSold,PctOfTotalPounds
Olive Oil - Extra Virgin ORG,Oils,Grocery,45039.15,0.034412,6536.8858,0.032550
Coffee - The Bean Whole,The Bean Whole,Local Love,35283.34,0.026958,2220.4735,0.011056
CRU Kombucha,Kombucha,Local Love,28571.28,0.021830,6613.7230,0.032933
Raw Cashews ORG,Nuts,Grocery,23603.48,0.018034,2109.3352,0.010503
Coffee - Hugo Coffee,Hugo Coffee,Local Love,23451.72,0.017918,1688.3915,0.008407
Maple Syrup ORG,Sweeteners,Grocery,22980.91,0.017558,2233.3252,0.011120
Raw Almonds ORG,Nuts,Grocery,20475.71,0.015644,1675.5883,0.008343
Eggs ORG,Dairy,Grocery,20091.05,0.015350,0.0000,0.000000
Sapadilla Hand Soap,Body Cleansing,Body Care,19389.01,0.014814,3188.9807,0.015879
Milk Honey Yogurt,Milk Honey Yogurt,Local Love,18611.99,0.014220,3159.9311,0.015735


#### Top Selling Products by Pounds Sold

In [11]:
SELECT TOP(20)
    sc.Subcategory,
    c.Category,
    p.ProductID,
    p.ProductName,
    SUM(l.PoundsSold) AS PoundsSold,
    SUM(l.PoundsSold)/ (SELECT SUM(PoundsSold) FROM SalesOrderLines) AS PctofTotal

FROM SalesOrderLines AS l
    INNER JOIN DimProducts AS p
    ON l.ProductID = p.ProductID
    INNER JOIN DimSubcategories AS sc
    ON p.SubcategoryID = sc.SubcategoryID
    INNER JOIN DimCategories AS c
    ON sc.CategoryID = c.CategoryID

WHERE p.PricingType = 'PerPound'

GROUP BY sc.Subcategory, c.Category, p.ProductID, p.ProductName
ORDER BY PoundsSold DESC

(20 rows affected)

Total execution time: 00:00:00.166

Subcategory,Category,ProductID,ProductName,PoundsSold,PctofTotal
Laundry Cleaners,Cleaning Supply,76,EcosPro Laundry Detergent - Magnolia Lily,7921.0077,0.039442
Kombucha,Local Love,110,CRU Kombucha,6613.7230,0.032933
Oils,Grocery,149,Olive Oil - Extra Virgin ORG,6536.8858,0.032550
Sweeteners,Grocery,42,Cane Sugar ORG,6311.1972,0.031426
Dishsoaps,Cleaning Supply,78,EcosPro Dish Soap - Hand Washing,5998.5351,0.029869
Multi Purpose,Cleaning Supply,5,White Vinegar - 10% acidity (Cleaning),5800.2368,0.028882
Laundry Cleaners,Cleaning Supply,88,EcosPro Laundry Detergent - Free & Clear,5048.9640,0.025141
Seeds And Grains,Grocery,39,Rolled Oats ORG,4660.0017,0.023204
Other Cleaners,Cleaning Supply,1064,Cleaner Bombs - Toilet - Cat - Etc,3723.0000,0.018538
Body Cleansing,Body Care,862,Sapadilla Hand Soap,3188.9807,0.015879


## Average Order Value

#### Total Average Order Value by Month and Moving 3 Month Average (Includes Retail and Wholesale)

In [12]:
SELECT 
    EOMONTH(o.OrderDate) AS Month,
    COUNT(DISTINCT o.OrderID) AS OrderCount,
    SUM(l.SaleAmount) AS Amount,
    SUM(l.SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue,
    AVG(SUM(l.SaleAmount)/COUNT(DISTINCT o.OrderID)) OVER(
        ORDER BY EOMONTH(o.OrderDate) -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMoAvgAOV
FROM
    SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID

GROUP BY EOMONTH(o.OrderDate)
ORDER BY EOMONTH(o.OrderDate)

(35 rows affected)

Total execution time: 00:00:02.925

Month,OrderCount,Amount,AvgOrderValue,MovingThreeMoAvgAOV
2019-02-28,749,14964.79,19.979692,19.979692
2019-03-31,1003,20949.45,20.886789,20.433240
2019-04-30,899,17287.15,19.229310,20.031930
2019-05-31,1184,24424.98,20.629206,20.248435
2019-06-30,1137,27454.37,24.146323,21.334946
2019-07-31,1120,27423.94,24.485660,23.087063
2019-08-31,1095,26306.96,24.024621,24.218868
2019-09-30,1109,28842.81,26.007944,24.839408
2019-10-31,1135,29368.16,25.875030,25.302531
2019-11-30,1153,29010.85,25.161188,25.681387


#### Retail AOV and Three Month Moving Average


In [13]:
SELECT 
    EOMONTH(o.OrderDate) AS Month,
    COUNT(DISTINCT o.OrderID) AS OrderCount,
    SUM(l.SaleAmount) AS Amount,
    SUM(l.SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue,
    AVG(SUM(l.SaleAmount)/COUNT(DISTINCT o.OrderID)) OVER(
        ORDER BY EOMONTH(o.OrderDate) -- Order the records by transaction month, ensuring the average is calculated chronologically
        ROWS BETWEEN 2 PRECEDING and CURRENT ROW -- specifies the range of rows to include in the average as 2 preceeding and current row
    ) AS MovingThreeMoAvgAOV
FROM
    SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID
    INNER JOIN dimCustomers AS c
    ON o.CustomerID = c.CustomerID

WHERE c.CustomerType = 'Retail'

GROUP BY EOMONTH(o.OrderDate)
ORDER BY EOMONTH(o.OrderDate)

(35 rows affected)

Total execution time: 00:00:01.074

Month,OrderCount,Amount,AvgOrderValue,MovingThreeMoAvgAOV
2019-02-28,747,14561.97,19.493935,19.493935
2019-03-31,998,19753.03,19.792615,19.643275
2019-04-30,895,16768.16,18.735374,19.340641
2019-05-31,1177,23642.95,20.087468,19.538485
2019-06-30,1126,24337.25,21.613898,20.145580
2019-07-31,1105,23242.34,21.033791,20.911719
2019-08-31,1088,24639.41,22.646516,21.764735
2019-09-30,1096,25514.16,23.279343,22.319883
2019-10-31,1123,26210.13,23.339385,23.088414
2019-11-30,1144,26922.07,23.533277,23.384001


## Customer Analysis

#### Compare Retail and Wholesale Sales Statistics


In [3]:
SELECT
    c.CustomerType,
    SUM(SaleAmount) AS Sales,
    SUM(SaleAmount)/
    (SELECT 
        SUM(SaleAmount)
        FROM SalesOrderLines) AS PctOfSales, --Use a subquery to calculate total sales and percent of total sales
    COUNT(DISTINCT o.OrderID) As TransactionCount,
    SUM(SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue,
    COUNT(DISTINCT o.CustomerID) AS UniqueCustomerIDs

FROM SalesOrderLines AS l
    INNER JOIN SalesOrders AS o
    ON l.OrderID = o.OrderID   
    INNER JOIN DimCustomers AS c
    ON o.CustomerID = c.CustomerID

GROUP BY c.CustomerType

(2 rows affected)

Total execution time: 00:00:01.073

CustomerType,Sales,PctOfSales,TransactionCount,AvgOrderValue,UniqueCustomerIDs
Wholesale,57256.92,0.043747,230,248.943130,15
Retail,1251542.46,0.956252,41220,30.362505,3917


#### Retail Sales: Identified Customers vs Anonymous Sales - CASE Statement and CTE to group by CASE Statement

Sales Composition (% of Total), AOV, Unique Customers, Order Count

In [4]:
-- Create a CTE of Sales Transactions, adding a LoyaltyType using a CASE statement
WITH CustomerSales AS(
    SELECT
        o.CustomerID,
        
        CASE 
            WHEN o.CustomerID = 'Unknown' THEN 'Anonymous'
            ELSE 'CustomerIDs'
            END AS LoyaltyType,
        
        l.SaleAmount,
        o.OrderID AS OrderID
    
    FROM SalesOrderLines AS l
        INNER JOIN SalesOrders AS o
        ON l.OrderID = o.OrderID   
        INNER JOIN DimCustomers AS c
        ON o.CustomerID = c.CustomerID

    WHERE c.CustomerType = 'Retail'
)

-- Create a query subtotaling by the newly assigned LoyaltyType
SELECT
    LoyaltyType,
    SUM(SaleAmount) AS Sales,
    SUM(SaleAmount)/
    (SELECT 
        SUM(SaleAmount)
        FROM CustomerSales) AS PctOfSales, --Use a subquery to calculate total sales and percent of total sales
    COUNT(DISTINCT OrderID) As OrderCount,
    SUM(SaleAmount) / COUNT(DISTINCT OrderID) AS AvgOrderValue,
    COUNT(DISTINCT CustomerID) AS UniqueCustomerIDs
FROM CustomerSales
GROUP BY LoyaltyType

(2 rows affected)

Total execution time: 00:00:01.062

LoyaltyType,Sales,PctOfSales,OrderCount,AvgOrderValue,UniqueCustomerIDs
Anonymous,459830.35,0.367410,18643,24.665040,1
CustomerIDs,791712.11,0.632589,22577,35.067197,3916


## Day of Week Sales

#### Avg Transaction Count, Daily Sales and AOV by Day of the Week - Retails Sales

In [21]:
-- Create a CTE of Daily Sales
WITH Daily AS(
    SELECT
        o.OrderDate,
        DATENAME(WEEKDAY,o.OrderDate) AS DayofWeek,
        DATEPART(WEEKDAY, o.OrderDate) AS DayNumber,
        COUNT(DISTINCT o.OrderID) AS DailyTransactions,
        SUM(l.SaleAmount) AS DailySales,
        SUM(l.SaleAmount) / COUNT(DISTINCT o.OrderID) AS AvgOrderValue
    FROM
        SalesOrderLines AS l
        INNER JOIN SalesOrders AS o
        ON l.OrderID = o.OrderID
        INNER JOIN DimCustomers AS c
        ON o.CustomerID = c.CustomerID

    WHERE c.CustomerType = 'Retail'
    
    GROUP BY o.OrderDate 
)

-- Summarize Sales Data by Day of Week
SELECT
    DayofWeek,
    Sum(DailyTransactions) AS TotalTransactionCount,
    AVG(DailyTransactions) AS AvgDailyTransactions,
    Sum(DailySales) AS TotalSales,
    Avg(DailySales) AS AvgDailySales,
    SUM(DailySales) / Sum(DailyTransactions) AS AvgOrderValue
FROM Daily
GROUP BY DayofWeek, DayNumber
ORDER BY DayNumber

(7 rows affected)

Total execution time: 00:00:00.351

DayofWeek,TotalTransactionCount,AvgDailyTransactions,TotalSales,AvgDailySales,AvgOrderValue
Sunday,5941,40,189939.19,1292.103333,31.970912
Monday,5588,38,165201.91,1139.323517,29.563691
Tuesday,5228,35,154732.23,1038.471342,29.596830
Wednesday,5020,33,145338.16,982.014594,28.951824
Thursday,4655,33,134008.38,964.089064,28.788051
Friday,5616,38,175745.27,1203.734726,31.293673
Saturday,9172,61,286577.32,1923.337718,31.244801
